In [ ]:
import os
import sys
import tensorflow as tf
import time
import pandas as pd
import numpy as np 
from matplotlib import pyplot as plt
from scipy.interpolate import interp1d
from mpl_toolkits.mplot3d import Axes3D
from multiprocessing import Process
import fresnel

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"

# define method to calculate A_matrix

In [ ]:
def obtain_Amatrix2(r,k,alpha_j):
    #matrix operation to create A_matrix
    #reshape r
    r2=tf.reshape(r,(-1,1,1,3))
    r3=tf.reshape(r,(1,-1,1,3))
    #calculate the point difference
    r4=(r2-r3)
    #calculate the distance
    rij=tf.reshape(tf.norm(r4,axis=(2,3))+tf.eye(len(r),dtype=tf.complex64),(len(r),len(r),1,1))
    #normalize the distance
    r4_norm=r4/rij
    #calculate the 3*3 matrix
    r5=tf.matmul(tf.linalg.matrix_transpose(r4_norm),r4_norm)
    #read in wavenumber
    norm_k=tf.cast(tf.norm(k),tf.complex64)
    #convert types
    rij=tf.cast(rij,tf.complex64)
    r5=tf.cast(r5,tf.complex64)
    #calculate A according to Green function
    A_matrix=tf.exp(1j*norm_k*rij)/rij*(
    ((norm_k)**2)*(r5-tf.eye(3,dtype=tf.complex64))+(1j*norm_k*rij-1)/(rij**2)*(3*r5-tf.eye(3,dtype=tf.complex64)))
    #reshape A into two-dimensional matrix
    A_matrix=tf.reshape(tf.transpose(A_matrix,(1,3,0,2)),(len(r)*3,len(r)*3))
    #vanish the diagonal element and add the 1/alpha_j
    A_matrix=tf.linalg.set_diag(A_matrix, tf.zeros(A_matrix.shape[0],dtype=tf.complex64), name=None)+1/alpha_j*tf.eye(A_matrix.shape[0],dtype=tf.complex64)
    return A_matrix

In [ ]:
def calculate_r(d,N):
    return ((d**3)*N*3/4/np.pi)**(1/3)

In [ ]:
def obtain_rotation_matrix(theta_x,theta_y,theta_z):
    
    rot_x = tf.constant([[1,0,0],
        [0,np.cos(theta_x),-np.sin(theta_x)],
        [0,np.sin(theta_x),np.cos(theta_x)]],dtype=tf.complex64)

    rot_y = tf.constant([[np.cos(theta_y),0,np.sin(theta_y)],
        [0,1,0],
        [-np.sin(theta_y),0,np.cos(theta_y)]],dtype=tf.complex64)

    rot_z = tf.constant([[np.cos(theta_z),-np.sin(theta_z),0],
        [np.sin(theta_z),np.cos(theta_z),0],
        [0,0,1]],dtype=tf.complex64)

    rot_matrix = tf.matmul(rot_z, tf.matmul(rot_y, rot_x))
    return rot_matrix

In [ ]:
def delete_1D(new_position,position):
    A=np.array(np.around(new_position,7)).tolist()
    B=np.array(np.around(position,7)).tolist()
    A = [i for i in A if i not in B]
    return A

In [ ]:
def dense_to_sparse(x):
    idx = tf.where(tf.not_equal(x, 0))
    sparse = tf.SparseTensor(idx, tf.gather_nd(x, idx), x.get_shape())
    return sparse

# Read in the dielectric constant and simulate on the corrsponding wavelength

In [ ]:
diename='Au2'

In [ ]:
data = open(diename, "r")
diel_txt=[]
for x in data:
    diel_txt.append(x)
diel=[]
for x in diel_txt[3:]:
    diel.append([float(i) for i in x.replace('\n','').split('   ')])
diel=np.array(diel)
wavelength=diel[:,0]

In [ ]:
n_real = interp1d(wavelength, diel[:,1], kind='cubic')
n_image = interp1d(wavelength, diel[:,2], kind='cubic')

In [ ]:
plt.plot(np.linspace(0.3,1,100),n_real(np.linspace(0.3,1,100)))
plt.scatter(wavelength,diel[:,1])

In [ ]:
plt.plot(np.linspace(0.3,1,100),n_image(np.linspace(0.3,1,100)))
plt.scatter(wavelength,diel[:,2])

In [ ]:
plt.plot(np.linspace(0.3,1,100),n_real(np.linspace(0.3,1,100)))
plt.scatter(wavelength,diel[:,1])

In [ ]:
wavelength_exp=np.linspace(0.4,0.95,56)

In [ ]:
#polarization
refractive_target=(n_real(wavelength_exp)+1j*n_image(wavelength_exp)).astype("complex64") #refractive index of target
refractive_medium=1.333 #refractive index of ambient medium
refractive_rel=refractive_target/refractive_medium #relative refractive index
wavelength_rel=wavelength_exp/refractive_medium #relative wavelength
k=2*np.pi/wavelength_rel #wavenumber in micron meter

#fixed cell space
d=1/1000
#define Clausius-Mossotti polarizability and FLTRDD polarizability further
alpha_CM=3*(d**3)*(refractive_rel**2-1)/4/np.pi/(refractive_rel**2+2)
D=alpha_CM/(d**3)*(4/3*((k*d)**2)+2/3/np.pi*np.log((np.pi-k*d)/(np.pi+k*d))*((k*d)**3)+2j/3*((k*d)**3))
alpha_j=alpha_CM/(1+D)

# A Simple test for the first wavelength

In [ ]:
position=[]
R= 5.5
for x in range(-10,10):
    for y in range(-10,10):
        for z in range(-17,16):
            if x**2 + y**2 <=R**2:
                position.append([x,y,z]) 
position=np.array(position)
len(position)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(position[:,0], position[:,1], position[:,2])

In [ ]:
scene = fresnel.Scene()
geometry1 = fresnel.geometry.Sphere(scene, N=len(position), radius=0.5)
geometry1.position[:] = position
geometry1.material = fresnel.material.Material(color=fresnel.color.linear([0.25,0.5,0.9]),
                                              roughness=0.8)
fresnel.preview(scene)

In [ ]:
r_old=d*tf.constant(position,dtype=tf.complex64)

In [ ]:
C_cross_total=[]
# C_cross_absorption=[]
for index_wave in range(len(wavelength_exp)):
    C_cross=[]
    print(index_wave)
    with tf.device('/GPU:0'):
        A_matrix1= obtain_Amatrix2(r_old,k[index_wave],alpha_j[index_wave])
        A_matrix_reverse=tf.linalg.inv(A_matrix1, adjoint=False, name=None)
        A_matrix1_shape=A_matrix1.shape[0]
        A_matrix1=[]
        
    with tf.device('/GPU:0'):
        #set different orientations
        rotation_matrix_incident=[]
        rotation_matrix_polarization=[]
        for theta_y in np.arccos(np.linspace(1,-1,10)):
            for theta_z in np.linspace(0,2*np.pi,10,endpoint=False):
                for theta_z_pol in np.linspace(0,2*np.pi,10,endpoint=False):
                    rotation_matrix_incident.append(obtain_rotation_matrix(0,theta_y,theta_z))
                    rotation_matrix_polarization.append(obtain_rotation_matrix(0,0,theta_z_pol))

        rotation_matrix_incident=tf.stack(rotation_matrix_incident)
        rotation_matrix_polarization=tf.stack(rotation_matrix_polarization)

        #set the initial electric field
        E_in0=tf.constant([[1,0,0]],dtype=tf.complex64)
        E_pol=tf.constant([[0],[0],[-k[index_wave]]],dtype=tf.complex64)

        #rotate the initial electric field
        E_pol_set=tf.matmul(rotation_matrix_incident,E_pol)
        E_in0_set=tf.matmul(rotation_matrix_polarization,tf.transpose(E_in0))
        E_in0_set=tf.linalg.matrix_transpose(tf.matmul(rotation_matrix_incident,E_in0_set))


        #calculate k dot r
        kdotr=tf.matmul(r_old,E_pol_set)

        #calculate the E_j for every dipole
        E_j=tf.exp(1j*kdotr)*E_in0_set
        E_j=tf.reshape(E_j,(E_j.shape[0],-1,1))
        E_j=tf.cast(E_j,tf.complex64)

        #calculate the dipoles
        dipoles=tf.matmul(A_matrix_reverse,E_j)
        
        C_cross_temp=4*np.pi*k[index_wave]*tf.reduce_sum(tf.math.imag(tf.math.conj(E_j)*dipoles))/E_j.shape[0]
# #         C_cross_absorption_temp=4*np.pi*k[index_wave]*tf.reduce_sum((tf.math.imag(tf.cast(tf.math.conj((1/alpha_j[index_wave])),tf.complex64)*dipoles*tf.math.conj((dipoles)))
#                                                        -2/3*(k[index_wave]**3)*abs(dipoles)))
#         C_cross_absorption.append(C_cross_absorption_temp)
        print(C_cross_temp)
        C_cross.append(C_cross_temp)
        
    C_cross_total.append(C_cross)

In [ ]:
result=np.array(C_cross_total)/np.pi/calculate_r(d,len(position))**2

In [ ]:
n_exction = interp1d(wavelength_exp,result.flatten(),"cubic")

In [ ]:
plt.plot(np.linspace(0.4,0.95,1101),n_exction(np.linspace(0.4,0.95,1101)))

In [ ]:
np.linspace(0.4,0.95,1101)[np.argmax(n_exction(np.linspace(0.4,0.95,1101)))]

In [ ]:
data = result
np.savetxt('11_by_33nm_rods.csv',data,delimiter=',')